In [19]:
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize
import ibm_boto3
from ibm_botocore.client import Config, ClientError
import time
import numpy as np
from io import BytesIO
import urllib.request as req
import wget
from PIL import Image
from datetime import datetime
import io as libio

In [5]:
cities_df = pd.read_csv('cities_euro.csv')
cities_df

,City,AccuName,Country,Population,Latitude,Longitude,Geoposition,APICode
0,LONDON,London,UK,"7,074,000",51.507351,-0.127758,"51.507351,-0.127758",51.507351%2C-0.127758
1,BERLIN,Berlin,Germany,"3,387,000",52.520008,13.404954,"52.520008,13.404954",52.520008%2C13.404954
2,MADRID,Madrid,Spain,"2,824,000",40.416775,-3.703790,"40.416775,-3.70379",40.416775%2C-3.70379
3,ROMA,ROMA,Italy,"2,649,000",41.902782,12.496365,"41.902782,12.496365",41.902782%2C12.496365
4,KIEV,KIEV,Ukraine,"2,590,000",50.450100,30.523399,"50.4501,30.523399",50.4501%2C30.523399
5,PARIS,PARIS,France,"2,152,000",48.856613,2.352222,"48.856613,2.352222",48.856613%2C2.352222
6,BUCURESTI (Bucharest),BUCURESTI (Bucharest),Romania,"2,016,000",46.195992,21.287560,"46.195992,21.28756",46.195992%2C21.28756
7,BUDAPEST,BUDAPEST,Hungary,"1,825,000",47.497913,19.040236,"47.497913,19.040236",47.497913%2C19.040236
8,Hamburg,Hamburg,Germany,"1,705,000",53.551086,9.993682,"53.551086,9.993682",53.551086%2C9.993682
9,MINSK,MINSK,Belarus,"1,677,000",53.904541,27.561523,"53.904541,27.561523",53.904541%2C27.561523


In [8]:
location = requests.get("http://dataservice.accuweather.com/locations/v1/cities/geoposition/search?apikey=9vvRWrL3ioBZFzTcuXg1QPZz0f1Xssu0&q=52.520008%2C13.404954")

In [9]:
location_df = json_normalize(location.json())

In [10]:
location_df

,AdministrativeArea.CountryID,AdministrativeArea.EnglishName,AdministrativeArea.EnglishType,AdministrativeArea.ID,AdministrativeArea.Level,AdministrativeArea.LocalizedName,AdministrativeArea.LocalizedType,Country.EnglishName,Country.ID,Country.LocalizedName,...,Region.ID,Region.LocalizedName,SupplementalAdminAreas,TimeZone.Code,TimeZone.GmtOffset,TimeZone.IsDaylightSaving,TimeZone.Name,TimeZone.NextOffsetChange,Type,Version
0,DE,Berlin,State,BE,1,Berlin,State,Germany,DE,Germany,...,EUR,Europe,"[{'Level': 3, 'LocalizedName': 'Berlin', 'Engl...",CET,1.0,False,Europe/Berlin,2020-03-29T01:00:00Z,City,1


In [11]:
location_columns = list(location_df.columns) 

In [12]:
location_columns

['AdministrativeArea.CountryID',
 'AdministrativeArea.EnglishName',
 'AdministrativeArea.EnglishType',
 'AdministrativeArea.ID',
 'AdministrativeArea.Level',
 'AdministrativeArea.LocalizedName',
 'AdministrativeArea.LocalizedType',
 'Country.EnglishName',
 'Country.ID',
 'Country.LocalizedName',
 'DataSets',
 'EnglishName',
 'GeoPosition.Elevation.Imperial.Unit',
 'GeoPosition.Elevation.Imperial.UnitType',
 'GeoPosition.Elevation.Imperial.Value',
 'GeoPosition.Elevation.Metric.Unit',
 'GeoPosition.Elevation.Metric.UnitType',
 'GeoPosition.Elevation.Metric.Value',
 'GeoPosition.Latitude',
 'GeoPosition.Longitude',
 'IsAlias',
 'Key',
 'LocalizedName',
 'ParentCity.EnglishName',
 'ParentCity.Key',
 'ParentCity.LocalizedName',
 'PrimaryPostalCode',
 'Rank',
 'Region.EnglishName',
 'Region.ID',
 'Region.LocalizedName',
 'SupplementalAdminAreas',
 'TimeZone.Code',
 'TimeZone.GmtOffset',
 'TimeZone.IsDaylightSaving',
 'TimeZone.Name',
 'TimeZone.NextOffsetChange',
 'Type',
 'Version']

In [13]:
for col in location_columns:
    cities_df[col] = ""

cities_df

,City,AccuName,Country,Population,Latitude,Longitude,Geoposition,APICode,AdministrativeArea.CountryID,AdministrativeArea.EnglishName,...,Region.ID,Region.LocalizedName,SupplementalAdminAreas,TimeZone.Code,TimeZone.GmtOffset,TimeZone.IsDaylightSaving,TimeZone.Name,TimeZone.NextOffsetChange,Type,Version
0,LONDON,London,UK,"7,074,000",51.507351,-0.127758,"51.507351,-0.127758",51.507351%2C-0.127758,,,...,,,,,,,,,,
1,BERLIN,Berlin,Germany,"3,387,000",52.520008,13.404954,"52.520008,13.404954",52.520008%2C13.404954,,,...,,,,,,,,,,
2,MADRID,Madrid,Spain,"2,824,000",40.416775,-3.703790,"40.416775,-3.70379",40.416775%2C-3.70379,,,...,,,,,,,,,,
3,ROMA,ROMA,Italy,"2,649,000",41.902782,12.496365,"41.902782,12.496365",41.902782%2C12.496365,,,...,,,,,,,,,,
4,KIEV,KIEV,Ukraine,"2,590,000",50.450100,30.523399,"50.4501,30.523399",50.4501%2C30.523399,,,...,,,,,,,,,,
5,PARIS,PARIS,France,"2,152,000",48.856613,2.352222,"48.856613,2.352222",48.856613%2C2.352222,,,...,,,,,,,,,,
6,BUCURESTI (Bucharest),BUCURESTI (Bucharest),Romania,"2,016,000",46.195992,21.287560,"46.195992,21.28756",46.195992%2C21.28756,,,...,,,,,,,,,,
7,BUDAPEST,BUDAPEST,Hungary,"1,825,000",47.497913,19.040236,"47.497913,19.040236",47.497913%2C19.040236,,,...,,,,,,,,,,
8,Hamburg,Hamburg,Germany,"1,705,000",53.551086,9.993682,"53.551086,9.993682",53.551086%2C9.993682,,,...,,,,,,,,,,
9,MINSK,MINSK,Belarus,"1,677,000",53.904541,27.561523,"53.904541,27.561523",53.904541%2C27.561523,,,...,,,,,,,,,,


In [14]:
for index, row in cities_df.iterrows():
    get_APICode = row['APICode']
    create_APIcall = "http://dataservice.accuweather.com/locations/v1/cities/geoposition/search?apikey=SnUPrdKxuZcUsSzXWDl5cae481d5hqNA&q=" + get_APICode
    location = requests.get(create_APIcall)
    location_df = json_normalize(location.json())
    for col in location_columns:
        if col in location_df.columns:
            cities_df.at[index,col] = location_df.at[0,col]

cities_df

,City,AccuName,Country,Population,Latitude,Longitude,Geoposition,APICode,AdministrativeArea.CountryID,AdministrativeArea.EnglishName,...,Region.ID,Region.LocalizedName,SupplementalAdminAreas,TimeZone.Code,TimeZone.GmtOffset,TimeZone.IsDaylightSaving,TimeZone.Name,TimeZone.NextOffsetChange,Type,Version
0,LONDON,London,UK,"7,074,000",51.507351,-0.127758,"51.507351,-0.127758",51.507351%2C-0.127758,GB,Westminster,...,EUR,Europe,"[{'Level': 0, 'LocalizedName': 'England', 'Eng...",GMT,0,False,Europe/London,2020-03-29T01:00:00Z,City,1
1,BERLIN,Berlin,Germany,"3,387,000",52.520008,13.404954,"52.520008,13.404954",52.520008%2C13.404954,DE,Berlin,...,EUR,Europe,"[{'Level': 3, 'LocalizedName': 'Berlin', 'Engl...",CET,1,False,Europe/Berlin,2020-03-29T01:00:00Z,City,1
2,MADRID,Madrid,Spain,"2,824,000",40.416775,-3.703790,"40.416775,-3.70379",40.416775%2C-3.70379,ES,Madrid,...,EUR,Europe,[],CET,1,False,Europe/Madrid,2020-03-29T01:00:00Z,City,1
3,ROMA,ROMA,Italy,"2,649,000",41.902782,12.496365,"41.902782,12.496365",41.902782%2C12.496365,IT,Lazio,...,EUR,Europe,"[{'Level': 2, 'LocalizedName': 'Roma', 'Englis...",CET,1,False,Europe/Rome,2020-03-29T01:00:00Z,City,1
4,KIEV,KIEV,Ukraine,"2,590,000",50.450100,30.523399,"50.4501,30.523399",50.4501%2C30.523399,UA,Kiev City,...,EUR,Europe,[],EET,2,False,Europe/Kiev,2020-03-29T01:00:00Z,City,1
5,PARIS,PARIS,France,"2,152,000",48.856613,2.352222,"48.856613,2.352222",48.856613%2C2.352222,FR,Ville de Paris,...,EUR,Europe,"[{'Level': 2, 'LocalizedName': 'Paris', 'Engli...",CET,1,False,Europe/Paris,2020-03-29T01:00:00Z,City,1
6,BUCURESTI (Bucharest),BUCURESTI (Bucharest),Romania,"2,016,000",46.195992,21.287560,"46.195992,21.28756",46.195992%2C21.28756,RO,Arad,...,EUR,Europe,[],EET,2,False,Europe/Bucharest,2020-03-29T01:00:00Z,City,1
7,BUDAPEST,BUDAPEST,Hungary,"1,825,000",47.497913,19.040236,"47.497913,19.040236",47.497913%2C19.040236,HU,Budapest,...,EUR,Europe,[],CET,1,False,Europe/Budapest,2020-03-29T01:00:00Z,City,1
8,Hamburg,Hamburg,Germany,"1,705,000",53.551086,9.993682,"53.551086,9.993682",53.551086%2C9.993682,DE,Hamburg,...,EUR,Europe,"[{'Level': 3, 'LocalizedName': 'Hamburg', 'Eng...",CET,1,False,Europe/Berlin,2020-03-29T01:00:00Z,City,1
9,MINSK,MINSK,Belarus,"1,677,000",53.904541,27.561523,"53.904541,27.561523",53.904541%2C27.561523,BY,Minsk City,...,EUR,Europe,[],MSK,3,False,Europe/Minsk,None,City,1


In [22]:
cities_df.to_csv('cities_allinfo.csv')

C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning
C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning
C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: Deprecati

C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning
C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning
C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: Deprecati

C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning
C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning
C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: Deprecati

C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning
C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning
C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: Deprecati

C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning
C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning
C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: Deprecati

...

C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning
C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning
C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: Deprecati

C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning
C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning
C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: Deprecati

C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning
C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning
C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: Deprecati

C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning
C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning
C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: Deprecati

C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning
C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning
C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: Deprecati

In [61]:
current = requests.get("http://dataservice.accuweather.com/currentconditions/v1/2532685?apikey=9vvRWrL3ioBZFzTcuXg1QPZz0f1Xssu0")
images = requests.get("http://dataservice.accuweather.com/imagery/v1/maps/radsat/1024x1024/2532685?apikey=SnUPrdKxuZcUsSzXWDl5cae481d5hqNA")
jprint(current.json())
jprint(images.json())

[
    {
        "EpochTime": 1574452860,
        "HasPrecipitation": false,
        "IsDayTime": false,
        "Link": "http://www.accuweather.com/en/gb/embankment/sw1a-2/current-weather/2532685?lang=en-us",
        "LocalObservationDateTime": "2019-11-22T20:01:00+00:00",
        "MobileLink": "http://m.accuweather.com/en/gb/embankment/sw1a-2/current-weather/2532685?lang=en-us",
        "PrecipitationType": null,
        "Temperature": {
            "Imperial": {
                "Unit": "F",
                "UnitType": 18,
                "Value": 46.0
            },
            "Metric": {
                "Unit": "C",
                "UnitType": 17,
                "Value": 7.9
            }
        },
        "WeatherIcon": 7,
        "WeatherText": "Cloudy"
    }
]
{
    "Link": "http://www.accuweather.com/en/gb/national/satellite?lang=en-us",
    "MobileLink": "http://m.accuweather.com/en/gb/national/satellite?lang=en-us",
    "Satellite": {
        "Images": [
            {
     

In [62]:
current = requests.get("http://dataservice.accuweather.com/currentconditions/v1/328328?apikey=9vvRWrL3ioBZFzTcuXg1QPZz0f1Xssu0")
images = requests.get("http://dataservice.accuweather.com/imagery/v1/maps/radsat/1024x1024/328328?apikey=SnUPrdKxuZcUsSzXWDl5cae481d5hqNA")
jprint(current.json())
jprint(images.json())

[
    {
        "EpochTime": 1574452860,
        "HasPrecipitation": false,
        "IsDayTime": false,
        "Link": "http://www.accuweather.com/en/gb/london/ec4a-2/current-weather/328328?lang=en-us",
        "LocalObservationDateTime": "2019-11-22T20:01:00+00:00",
        "MobileLink": "http://m.accuweather.com/en/gb/london/ec4a-2/current-weather/328328?lang=en-us",
        "PrecipitationType": null,
        "Temperature": {
            "Imperial": {
                "Unit": "F",
                "UnitType": 18,
                "Value": 46.0
            },
            "Metric": {
                "Unit": "C",
                "UnitType": 17,
                "Value": 7.9
            }
        },
        "WeatherIcon": 7,
        "WeatherText": "Cloudy"
    }
]
{
    "Link": "http://www.accuweather.com/en/gb/national/satellite?lang=en-us",
    "MobileLink": "http://m.accuweather.com/en/gb/national/satellite?lang=en-us",
    "Radar": {
        "Images": [
            {
                "Da

In [90]:
# Constants for IBM COS values
COS_ENDPOINT = "https://s3.eu-gb.cloud-object-storage.appdomain.cloud"
COS_AUTH_ENDPOINT = "https://iam.cloud.ibm.com/identity/token"
COS_API_KEY_ID = "BD98N-vqonh983XVsKzNhIRx62oKMGZDodhNTL4ou9Tp" 
COS_RESOURCE_CRN = "crn:v1:bluemix:public:cloud-object-storage:global:a/5654a1f2157d42d482817601f6b4f60c:1b9a82e2-2215-4c79-ac53-7f021a893306:bucket:cos.deeplearning.walekova" 
COS_BUCKET = 'accu-clouds'

cos = ibm_boto3.client("s3",
    ibm_api_key_id=COS_API_KEY_ID,
    ibm_service_instance_id=COS_RESOURCE_CRN,
    ibm_auth_endpoint=COS_AUTH_ENDPOINT,
    config=Config(signature_version="oauth"),
    endpoint_url=COS_ENDPOINT
)
   
    
def writeImage(req,tod,typ):
    
    request = "http://vortex.accuweather.com/adc2004/mapspace/"+ req +"/1024x1024/EUR3_19"+ tod +"00.png"

    req_str = request[-14:]

    response = requests.get(request)
    img = Image.open(BytesIO(response.content))

    # Create buffer to hold jpeg representation of image in 'io.BytesIO' object
    bufImage = libio.BytesIO()
    # Store jpeg representation of image in buffer
    img.save(bufImage,"PNG") 
    # Rewind the buffer to beginning
    bufImage.seek(0)

    file = typ + req_str

    # Provide the object to the Body parameter of put_object to write image to COS
    cos.put_object(Bucket= COS_BUCKET, Body = bufImage , Key = file, ContentType = 'image/png')

req = 'euradar'
typ = 'satellite_'
dayandtime = ['112116','112117','112118','112119','112120','112121','112122','112123',\
              '112200','112201','112202','112203','112204','112205','112206','112207',\
              '112208','112209','112210','112211','112212','112213','112214','112215',\
              '112216','112217','112218','112219','112220']
for tod in dayandtime:
    writeImage(req,tod,typ)

dayandtime = ['112116','112117','112118','112119','112120','112121','112122','112123',\
              '112200','112201','112202','112203','112204','112205',\
              '112208','112209','112210','112211','112212','112213','112214','112215',\
              '112216','112217','112218','112219','112220']
req = 'world_ir'
typ = 'radar_'    
for tod in dayandtime:
    writeImage(req,tod,typ)    

C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning
C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning
C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: Deprecati

In [97]:
history_col = ['EpochTime', 'HasPrecipitation', 'IsDayTime', 'Link', 'LocalObservationDateTime',\
               'MobileLink', 'PrecipitationType', 'Temperature.Imperial.Unit', 'Temperature.Imperial.UnitType',\
               'Temperature.Imperial.Value', 'Temperature.Metric.Unit', 'Temperature.Metric.UnitType',\
               'Temperature.Metric.Value', 'WeatherIcon', 'WeatherText', 'Location_Key']

weather_24 = pd.DataFrame(columns=history_col)

weather_24
    

,EpochTime,HasPrecipitation,IsDayTime,Link,LocalObservationDateTime,MobileLink,PrecipitationType,Temperature.Imperial.Unit,Temperature.Imperial.UnitType,Temperature.Imperial.Value,Temperature.Metric.Unit,Temperature.Metric.UnitType,Temperature.Metric.Value,WeatherIcon,WeatherText,Location_Key


In [100]:
for index, row in cities_df.iterrows():
    get_locationKey = row['Key']
    request = requests.get("http://dataservice.accuweather.com/currentconditions/v1/"+ get_locationKey +"/historical/24?apikey=SnUPrdKxuZcUsSzXWDl5cae481d5hqNA")
    history_df = json_normalize(request.json())    
    history_df['Location_Key'] = get_locationKey
    weather_24 = pd.concat([weather_24, history_df])


C:\Users\walek\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,Code,EpochTime,HasPrecipitation,IsDayTime,Link,LocalObservationDateTime,Location_Key,Message,MobileLink,PrecipitationType,...,Temperature.Imperial.Unit,Temperature.Imperial.UnitType,Temperature.Imperial.Value,Temperature.Metric.Unit,Temperature.Metric.UnitType,Temperature.Metric.Value,WeatherIcon,WeatherText,fault.detail.errorcode,fault.faultstring
0,NaN,1574456520,False,False,http://www.accuweather.com/en/gb/embankment/sw...,2019-11-22T21:02:00+00:00,2532685,NaN,http://m.accuweather.com/en/gb/embankment/sw1a...,None,...,F,18,46.0,C,17,7.7,7,Cloudy,NaN,NaN
1,NaN,1574452860,False,False,http://www.accuweather.com/en/gb/embankment/sw...,2019-11-22T20:01:00+00:00,2532685,NaN,http://m.accuweather.com/en/gb/embankment/sw1a...,None,...,F,18,46.0,C,17,7.9,7,Cloudy,NaN,NaN
2,NaN,1574449320,False,False,http://www.accuweather.com/en/gb/embankment/sw...,2019-11-22T19:02:00+00:00,2532685,NaN,http://m.accuweather.com/en/gb/embankment/sw1a...,None,...,F,18,47.0,C,17,8.2,7,Cloudy,NaN,NaN
3,NaN,1574445720,False,False,http://www.accuweather.com/en/gb/embankment/sw...,2019-11-22T18:02:00+00:00,2532685,NaN,http://m.accuweather.com/en/gb/embankment/sw1a...,None,...,F,18,48.0,C,17,8.9,7,Cloudy,NaN,NaN
4,NaN,1574442120,False,False,http://www.accuweather.com/en/gb/embankment/sw...,2019-11-22T17:02:00+00:00,2532685,NaN,http://m.accuweather.com/en/gb/embankment/sw1a...,None,...,F,18,48.0,C,17,8.7,7,Cloudy,NaN,NaN
5,NaN,1574438460,False,True,http://www.accuweather.com/en/gb/embankment/sw...,2019-11-22T16:01:00+00:00,2532685,NaN,http://m.accuweather.com/en/gb/embankment/sw1a...,None,...,F,18,49.0,C,17,9.4,6,Mostly cloudy,NaN,NaN
6,NaN,1574434860,False,True,http://www.accuweather.com/en/gb/embankment/sw...,2019-11-22T15:01:00+00:00,2532685,NaN,http://m.accuweather.com/en/gb/embankment/sw1a...,None,...,F,18,49.0,C,17,9.5,7,Cloudy,NaN,NaN
7,NaN,1574431260,False,True,http://www.accuweather.com/en/gb/embankment/sw...,2019-11-22T14:01:00+00:00,2532685,NaN,http://m.accuweather.com/en/gb/embankment/sw1a...,None,...,F,18,49.0,C,17,9.6,7,Cloudy,NaN,NaN
8,NaN,1574427720,False,True,http://www.accuweather.com/en/gb/embankment/sw...,2019-11-22T13:02:00+00:00,2532685,NaN,http://m.accuweather.com/en/gb/embankment/sw1a...,None,...,F,18,49.0,C,17,9.6,7,Cloudy,NaN,NaN
9,NaN,1574424180,False,True,http://www.accuweather.com/en/gb/embankment/sw...,2019-11-22T12:03:00+00:00,2532685,NaN,http://m.accuweather.com/en/gb/embankment/sw1a...,None,...,F,18,49.0,C,17,9.5,7,Cloudy,NaN,NaN


In [104]:
weather_24.to_csv(r'weather_24_191122.csv')

C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning
C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning
C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: Deprecati

In [ ]:
def writeCSV():

    CSV = Image.open(BytesIO(response.content))

    # Create buffer to hold jpeg representation of image in 'io.BytesIO' object
    bufCSV = libio.BytesIO()
    # Store csv representation of csv in buffer
    CSV.save(bufCSV,"csv") 
    # Rewind the buffer to beginning
    bufCSV.seek(0)

    file = 'weather_24_191122.csv'

    # Provide the object to the Body parameter of put_object to write image to COS
    cos.put_object(Bucket= COS_BUCKET, Body = bufCSV , Key = file, ContentType = 'csv')
    
writeCSV()

In [6]:
def writeImage(stamp, latitude, longitude, tod, typ, locationKey):
    
    request = ("https://tilecache.rainviewer.com/v2//radar/"+stamp+"/512/20/"+latitude+"/"+longitude+"/2/0_0.png")

    response = requests.get(request)
    img = Image.open(BytesIO(response.content))

    # Create buffer to hold jpeg representation of image in 'io.BytesIO' object
    bufImage = libio.BytesIO()
    # Store jpeg representation of image in buffer
    img.save(bufImage,"PNG") 
    # Rewind the buffer to beginning
    bufImage.seek(0)

    file = "rainview_" + locationKey + tod

    # Provide the object to the Body parameter of put_object to write image to COS
    cos.put_object(Bucket= COS_BUCKET, Body = bufImage , Key = file, ContentType = 'image/png')


dayandtime = ['112116','112117','112118','112119','112120','112121','112122','112123',\
              '112200','112201','112202','112203','112204','112205','112206','112207',\
              '112208','112209','112210','112211','112212','112213','112214','112215',\
              '112216','112217','112218','112219','112220']

timestamp = requests.get("https://tilecache.rainviewer.com/api/maps.json")

for index, row in cities_df.iterrows():
    Latitude = row['Latitude']
    Longitude = row['Longitude']
    locationKey = row['Key']
    for stamp in timestamp:
        writeImage(stamp, latitude, longitude, tod, typ, locationKey)


KeyError: 'Key'

In [116]:
timestamp = requests.get("https://tilecache.rainviewer.com/api/maps.json")

In [121]:
stamps = str(timestamp.json())
print(type(stamps))
print(stamps)

<class 'str'>
[1574454600, 1574455200, 1574455800, 1574456400, 1574457000, 1574457600, 1574458200, 1574458800, 1574459400, 1574460000, 1574460600, 1574461200, 1574461800]


C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning


In [21]:
# Constants for IBM COS values
COS_ENDPOINT = "https://s3.eu-gb.cloud-object-storage.appdomain.cloud"
COS_AUTH_ENDPOINT = "https://iam.cloud.ibm.com/identity/token"
COS_API_KEY_ID = "BD98N-vqonh983XVsKzNhIRx62oKMGZDodhNTL4ou9Tp" 
COS_RESOURCE_CRN = "crn:v1:bluemix:public:cloud-object-storage:global:a/5654a1f2157d42d482817601f6b4f60c:1b9a82e2-2215-4c79-ac53-7f021a893306:bucket:cos.deeplearning.walekova" 
COS_BUCKET = 'accu-clouds'

cos = ibm_boto3.client("s3",
    ibm_api_key_id=COS_API_KEY_ID,
    ibm_service_instance_id=COS_RESOURCE_CRN,
    ibm_auth_endpoint=COS_AUTH_ENDPOINT,
    config=Config(signature_version="oauth"),
    endpoint_url=COS_ENDPOINT
)
   

def writeImage(stamp, tod, latitude, longitude, locationKey, city):
    
    request = "https://tilecache.rainviewer.com/v2/radar/"+stamp+"/512/4/"+latitude+"/"+longitude+"/2/0_0.png"
      
    response = requests.get(request)
    img = Image.open(BytesIO(response.content))

    # Create buffer to hold jpeg representation of image in 'io.BytesIO' object
    bufImage = libio.BytesIO()
    # Store jpeg representation of image in buffer
    img.save(bufImage,"PNG") 
    # Rewind the buffer to beginning
    bufImage.seek(0)

    file = "rainview_" + city + "_" + locationKey + "_" + tod + ".png"

    # Provide the object to the Body parameter of put_object to write image to COS
    cos.put_object(Bucket= COS_BUCKET, Body = bufImage , Key = file, ContentType = 'image/png')

timestamp = requests.get("https://tilecache.rainviewer.com/api/maps.json")
stamps = str(timestamp.json())
stamps = stamps.strip("[")
stamps = stamps.strip("]")
stamps = stamps.split(", ")

for index, row in cities_df.iterrows():
    latitude = str(row['Latitude'])
    longitude = str(row['Longitude'])
    locationKey = row['Key']
    city = row['City']
    for stamp in stamps:
        timestamp = int(stamp)
        dt_object = datetime.fromtimestamp(timestamp)
        check = dt_object.strftime('%M')
        if check == '00':
            tod = dt_object.strftime('%y%m%d%H%M')
            writeImage(stamp, tod, latitude, longitude, locationKey, city)


C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning
